In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
train = pd.read_csv('C:/Users/anees/Documents/CS/MP/datasets/toxic comment classification/train.csv')
test = pd.read_csv('C:/Users/anees/Documents/CS/MP/datasets/toxic comment classification/test.csv')

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
train_text = train.comment_text
test_text = test.comment_text

In [5]:
all_text = pd.concat([train_text, test_text])

In [6]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

In [7]:
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [8]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [9]:
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

Compare three different sets of features: word, char and both.

In [18]:
train_features = train_word_features
test_features = test_word_features

In [19]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [20]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9571333395186751
CV score for class severe_toxic is 0.9848176933150371
CV score for class obscene is 0.9799024281297329
CV score for class threat is 0.9771991118768119
CV score for class insult is 0.9694680040479869
CV score for class identity_hate is 0.9672945519244637
Total CV score is 0.9726358548021179


In [21]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [22]:
train_features = train_char_features
test_features = test_char_features

In [23]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9551325658472908
CV score for class severe_toxic is 0.98449924750931
CV score for class obscene is 0.9721998743961571
CV score for class threat is 0.9709287213766071
CV score for class insult is 0.9687186695112057
CV score for class identity_hate is 0.9599383611868205
Total CV score is 0.9685695733045652


In [ ]:
submission.to_csv('submission_char.csv', index=False)

In [10]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [11]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [18]:
classifiers = []

In [19]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    classifiers.append(classifier)

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9692150368530504
CV score for class severe_toxic is 0.9875939186173371
CV score for class obscene is 0.9838697445342036
CV score for class threat is 0.9833764609448602
CV score for class insult is 0.9774263999804423
CV score for class identity_hate is 0.9739428046661723
Total CV score is 0.9792375197280848


In [ ]:
submission.to_csv('submission_comb.csv', index=False)

In [15]:
from sklearn.externals import joblib

In [20]:
i = 0
for classifier in classifiers:
    joblib.dump(classifier, "lr_char_" + str(i) + ".joblib")
    i+=1